## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
  if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-w

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATE
from InstructABSA.data_prep import ModelReadyData

In [5]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [14]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()

# Get the input text into the required format
bos_instruction_lap = """Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm. 
Positive example 1- input: I charge it at night and skip taking the cord with me because of the good battery life.
utput: battery life
Positive example 2- input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
output: features, iChat, Photobooth, garage band
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm. 
Positive example 1- input: With the great variety on the menu , I eat here often and never get bored.
output: menu
Positive example 2- input: Great food, good size menu, great service and an unpretensious setting.
output: food, service, ambience
Now complete the following example-
input: """
eos_insrtuction = ' \noutput:'
res_tr_df = model_data.create_data_in_ate_format(res_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_res, eos_insrtuction)
lap_tr_df = model_data.create_data_in_ate_format(lap_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_lap, eos_insrtuction)
res_te_df = model_data.create_data_in_ate_format(res_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_res, eos_insrtuction)
lap_te_df = model_data.create_data_in_ate_format(lap_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_lap, eos_insrtuction)

In [15]:
print(res_tr_df['text'][0] + ' ' + res_tr_df['labels'][0])

Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm. 
Positive example 1- input: With the great variety on the menu , I eat here often and never get bored.
output: menu
Positive example 2- input: Great food, good size menu, great service and an unpretensious setting.
output: food, service, ambience
Now complete the following example-
input: But the staff was so horrible to us. 
output: staff


In [16]:
# Experimentation
experiment_id = 'laptops'
experiment_name = 'laptops_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':8,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
# Train model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3045
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1524
  Number of trainable parameters = 247534848


Trainer device: cuda:0

Model training started ....


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.244746
2,0.585900,0.217957
3,0.286800,0.210399
4,0.224300,0.215461


***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8
***** Running Evaluation *****
  Num examples = 800
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints/config.json
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-laptops_instruct/checkpoints/generation_config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-laptops_instruct/che

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

try:
    model_trainer
except:
    model_trainer = None

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

# Get prediction labels - OOD Training set
ood_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - OOD Testing set
ood_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

In [21]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] = id_tr_pred_labels
id_te_df['pred_labels'] = id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] = ood_te_pred_labels

# Metrics
def get_f1_acc(dataframe, true_col, pred_col):
    total_pred = 0
    total_gt = 0
    tp = 0
    for gt, pred in zip(dataframe[true_col][:], dataframe[pred_col][:]):
        gt_list = gt.split(', ')
        pred_list = pred.split(', ')
        total_pred+=len(pred_list)
        total_gt+=len(gt_list)
        for gt_val in gt_list:
            for pred_val in pred_list:
                if pred_val in gt_val:
                    tp+=1
    p = tp/total_pred
    r = tp/total_gt
    return p, r, 2*p*r/(p+r)

print('Train precision:', get_f1_acc(id_tr_df, 'labels', 'pred_labels')[0])
print('Test precision:', get_f1_acc(id_te_df, 'labels', 'pred_labels')[0])
print('OOD Test precision:', get_f1_acc(ood_te_df, 'labels', 'pred_labels')[0])

print('Train recall:', get_f1_acc(id_tr_df, 'labels', 'pred_labels')[1])
print('Test recall:', get_f1_acc(id_te_df, 'labels', 'pred_labels')[1])
print('OOD Test recall:', get_f1_acc(ood_te_df, 'labels', 'pred_labels')[1])

print('Train F1:', get_f1_acc(id_tr_df, 'labels', 'pred_labels')[2])
print('Test F1:', get_f1_acc(id_te_df, 'labels', 'pred_labels')[2])
print('OOD Test F1:', get_f1_acc(ood_te_df, 'labels', 'pred_labels')[2])

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, ood_te_filename), index = False)

Train precision: 0.9559087413483722
Test precision: 0.9392156862745098
OOD Test precision: 0.7744774477447744
Train recall: 0.9524904214559387
Test recall: 0.9282945736434108
OOD Test recall: 0.5289256198347108
Train F1: 0.9541965199590583
Test F1: 0.9337231968810915
OOD Test F1: 0.6285714285714286
